In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.tanh)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.tanh)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.tanh)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_101")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=50,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=3000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model_101', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022AFCCE7240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INF

INFO:tensorflow:global_step/sec: 6.89748
INFO:tensorflow:probabilities = [[0.11182968 0.12005918 0.09034364 0.11560057 0.07746934 0.09112929
  0.09941943 0.09213068 0.09530185 0.10671639]
 [0.09233323 0.10326038 0.09609346 0.1111965  0.10515235 0.08156528
  0.09741195 0.10321809 0.09666636 0.11310241]
 [0.10933122 0.10194927 0.10961191 0.09476874 0.11103146 0.09497752
  0.10184339 0.09082513 0.09301637 0.09264484]
 [0.11455522 0.09858742 0.10997314 0.10146351 0.08885431 0.07933602
  0.1013283  0.10123693 0.10815317 0.09651203]
 [0.10582647 0.07993568 0.10626254 0.10987522 0.08579663 0.08042519
  0.09586075 0.10284547 0.13500464 0.09816734]
 [0.10674135 0.09545325 0.10395011 0.107622   0.11500863 0.0922787
  0.08720849 0.08336829 0.10466864 0.10370043]
 [0.11604489 0.10112511 0.09040162 0.11318076 0.09293483 0.08119839
  0.1313707  0.09046316 0.08561912 0.09766144]
 [0.15106463 0.09087034 0.09134146 0.09381321 0.095526   0.09345485
  0.10688549 0.0867812  0.07475984 0.11550295]
 [0.1225

INFO:tensorflow:global_step/sec: 7.07392
INFO:tensorflow:probabilities = [[0.10713538 0.10024478 0.09060941 0.14023542 0.09283937 0.08812811
  0.08596589 0.07907151 0.10093688 0.11483313]
 [0.1243633  0.10717104 0.1057188  0.11251031 0.09469185 0.0912829
  0.08338898 0.09377221 0.09540551 0.09169513]
 [0.1039296  0.08946686 0.09061113 0.12446994 0.08440358 0.09258303
  0.08721279 0.1300118  0.10493942 0.09237185]
 [0.12578773 0.08346263 0.10688256 0.12735942 0.08532827 0.0685776
  0.09959451 0.07041978 0.11406311 0.11852428]
 [0.11312702 0.11275328 0.09602019 0.09894495 0.08750068 0.08629937
  0.10376057 0.08770106 0.11159539 0.1022975 ]
 [0.09651849 0.09941087 0.09420612 0.11454436 0.09031849 0.08303426
  0.10655548 0.10412807 0.10400485 0.10727903]
 [0.09721175 0.098896   0.09970675 0.10690393 0.09108919 0.08364651
  0.09859493 0.09182926 0.11884306 0.11327855]
 [0.10866928 0.09648856 0.1290614  0.11878629 0.08695286 0.07525194
  0.09036332 0.0866114  0.09860659 0.10920837]
 [0.11995

INFO:tensorflow:global_step/sec: 7.02458
INFO:tensorflow:probabilities = [[0.1474883  0.08818933 0.09999792 0.09655765 0.08683955 0.1041856
  0.09486174 0.11157313 0.09462338 0.07568352]
 [0.09297106 0.12374955 0.11614273 0.09301457 0.09698599 0.0810423
  0.09051285 0.08669617 0.11380283 0.10508199]
 [0.0973494  0.07662662 0.11291035 0.10960007 0.11325579 0.07996746
  0.12726808 0.0840572  0.0926686  0.10629652]
 [0.12804683 0.08574668 0.07810596 0.11674473 0.09133384 0.11043354
  0.07326886 0.10776522 0.10300647 0.10554784]
 [0.10610511 0.10678208 0.09213194 0.13003781 0.07905645 0.07824625
  0.09856173 0.10072062 0.09924334 0.10911474]
 [0.14364134 0.06624931 0.09513485 0.10911528 0.10538567 0.09719808
  0.09472446 0.09636322 0.09998724 0.09220055]
 [0.10860151 0.09405573 0.14310467 0.10908794 0.0885985  0.08452105
  0.10560804 0.06494725 0.10650319 0.09497208]
 [0.1305675  0.09323248 0.12294067 0.06501741 0.09020044 0.08113445
  0.12086982 0.09786351 0.09829778 0.09987599]
 [0.09987

INFO:tensorflow:global_step/sec: 7.10365
INFO:tensorflow:probabilities = [[0.13269033 0.07738195 0.11059603 0.15340197 0.06819168 0.06879969
  0.0720055  0.10874908 0.10577352 0.10241023]
 [0.13157307 0.07482047 0.18728024 0.09756876 0.07206529 0.06851274
  0.10563324 0.09098729 0.0754521  0.09610672]
 [0.10639778 0.08383433 0.09104333 0.11896394 0.08774801 0.08379734
  0.12095061 0.07390889 0.11404317 0.11931259]
 [0.1073463  0.1192447  0.09919438 0.11187664 0.08447737 0.07574315
  0.11238283 0.09172312 0.09579529 0.10221628]
 [0.09809522 0.0953626  0.08743207 0.10872611 0.12125389 0.08341946
  0.09594709 0.11483921 0.08861181 0.1063126 ]
 [0.09987537 0.11096125 0.09685858 0.11765914 0.08483093 0.06724048
  0.06952489 0.14528784 0.11895458 0.08880699]
 [0.22506632 0.0628117  0.08397011 0.09644085 0.09464818 0.10505973
  0.06797656 0.0808988  0.10142682 0.0817009 ]
 [0.15203288 0.08303297 0.07006003 0.13042063 0.11399659 0.08494153
  0.08969885 0.0882274  0.09745613 0.09013299]
 [0.099

...

INFO:tensorflow:global_step/sec: 7.06598
INFO:tensorflow:probabilities = [[0.91350913 0.00005262 0.00051883 0.00223067 0.00019026 0.08195198
  0.0003765  0.00028362 0.00057939 0.00030724]
 [0.00133881 0.00229139 0.00129051 0.00042337 0.9231536  0.00771785
  0.01140282 0.00287574 0.01552257 0.03398337]
 [0.00533742 0.432245   0.28954887 0.03507333 0.04146422 0.03074994
  0.00379219 0.00394446 0.1510692  0.00677539]
 [0.00106247 0.7859238  0.11929305 0.02596043 0.00969756 0.00742328
  0.00352443 0.0101269  0.03116151 0.00582666]
 [0.00049909 0.00406921 0.00244212 0.00231402 0.48531458 0.01537004
  0.007243   0.02242387 0.08069376 0.37963036]
 [0.04377865 0.00002458 0.00253016 0.00082409 0.6999856  0.0089173
  0.04685015 0.00966825 0.00755057 0.17987056]
 [0.0047105  0.08989096 0.13855568 0.3865204  0.00212887 0.10397846
  0.07258021 0.00183207 0.19521627 0.0045865 ]
 [0.00121773 0.7307332  0.01840371 0.04773476 0.00911872 0.04103345
  0.05615576 0.00410304 0.05896683 0.03253292]
 [0.0027

INFO:tensorflow:global_step/sec: 7.08899
INFO:tensorflow:probabilities = [[0.00092307 0.00231021 0.03564052 0.00174689 0.23690684 0.01812833
  0.6756494  0.00163545 0.0091882  0.01787109]
 [0.06881894 0.01896823 0.01648716 0.05267059 0.0182923  0.509899
  0.03525129 0.0095242  0.2106258  0.05946245]
 [0.0000608  0.00074737 0.00023657 0.00757956 0.2658503  0.12095553
  0.00152996 0.0208167  0.04992404 0.53229916]
 [0.00002617 0.00025241 0.00024691 0.00121746 0.8321791  0.00531021
  0.0011584  0.00628549 0.00967571 0.14364809]
 [0.00667839 0.00740411 0.00445111 0.02842961 0.00434669 0.04278397
  0.00353658 0.0054389  0.8569828  0.03994787]
 [0.00907397 0.00079821 0.00007289 0.00897341 0.00755815 0.88799924
  0.00007334 0.0098321  0.03247833 0.04314044]
 [0.98584974 0.00000034 0.00221662 0.00011375 0.00011978 0.00470343
  0.00586224 0.00016571 0.00067695 0.00029144]
 [0.01585063 0.00053375 0.00411029 0.02736172 0.08511651 0.72769517
  0.00282225 0.00537273 0.08415376 0.04698314]
 [0.38482

INFO:tensorflow:Saving checkpoints for 3000 into /tmp/mnist_convnet_model_101\model.ckpt.
INFO:tensorflow:Loss for final step: 0.37962657.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-16-16:20:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model_101\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-16-16:20:29
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.8903, global_step = 3000, loss = 0.43203712
{'accuracy': 0.8903, 'loss': 0.43203712, 'global_step': 3000}


SystemExit: 

c:\users\ninad\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
